In [1]:
import numpy as np

import ngsolve 
from ngsolve.webgui import Draw
import netgen.geom2d

### The complex Ginzburg-Landau model on different meshes in ngsolve

There are various forms of the complex Ginzburg-Landau equation for a field $A \in \mathbb{C}$.

From Cross' book
$$\partial_t A = A + (1 + i c_1) \nabla^2 A - (1-ic_3) |A|^2 A$$

From https://codeinthehole.com/tutorial/index.html  (Winterbottom) 
$$\partial_t A = A + (1 + i \alpha) \nabla^2 A - (1+ i\beta) |A|^2 A$$

These two  are the same if you send $\alpha \to c_1$ and
 $\beta \to - c_3$. 

from Chat\'e's and Manneville's 1996  review https://arxiv.org/abs/1608.07519
$$\partial_t A = A + (1 + i b_1) \nabla^2 A - (b_3-i) |A|^2 A$$

The real Ginzburg-Landau equation is setting $c_1, c_3=0$ of the Cross form or $\alpha=\beta=0$ of the Winterbottom form.
$$\partial_t A = A  + \nabla^2A - |A|^2 A$$

To rescale space you can multiply the Laplacian term by a factor.

In these forms the domain typically has size a few hundred so that interesting phenomena is seen. 
It is somewhat tedious to convert between Chate's form and the other two, but an advantage of 
that form is that they have conveniently classified the different behaviors as a function of $b_1, b_3$. 

The PDE  is obeyed in the domain $\Omega$, 
but on the boundary  $\partial \Omega$ we set $\frac{\partial A}{\partial n} = 0 $. 
This is a Neumann boundary condition. 

The weak/variational form of the problem is 
\begin{align}
\int_\Omega \partial_t u\ w\ dx = \int_\Omega D_u (\Delta u) w \ dx + \int_\Omega g(u) \ w \ dx
\end{align}
with function $$g(u) = u-(1 + i\beta)|u|^2 u, \qquad {\rm or} \qquad  
g(u) = u - (b_3 - i ) |u|^2 u$$ depending upon which form of the equation one wants to use. 

The diffusion coefficient $D_u = (1 + i \alpha)$, 
and for all test functions $w \in \hat V$.  
We integrate by parts the terms that contain a Laplacian operator 
\begin{align}
\int_\Omega \partial_t u\ w \ dx =   - D_u  \int_\Omega \nabla u \nabla w \ dx 
+ D_u\int_{\partial \Omega} \nabla u\ w \ ds
+ \int_\Omega g(u) \ w \ dx 
\end{align}
With normal derivative of $u$ equal to zero on the boundary, the  boundary terms in the above
equations can be neglected.
The weak form should be obeyed for all test functions $w \in \hat V$ with $\hat V = H^1(\Omega)$.
Since we lack Dirichlet regions of the boundary, there is no additional condition on $\hat V$. 
 We are following section 2.2.1 of the Fenics book.

We use a discrete subspace for $\hat V$ and assume we have a nice basis for it. 
We write each term as if it were an operator acting on a vector $w$ (aka the test function) in this basis. 

We split each time step into two pieces.  First updating $u,v$ with an implicit Crank-Nicolson step and 
then we take a first order forward Eulerian step to take into account the function $g$. 

The Crank-Nicolson step for  $\partial_t u = F u  $  with $F$ a linear op is the following scheme 
\begin{align}
\partial_t u \sim \frac{u^{n+1} - u^n}{\Delta t} = \frac{1}{2} \left(F u^{n+1} + F u^n\right)
\end{align}
\begin{align}
\left( 1 - \frac{\Delta t F}{2 } \right) u^{n+1} = \left(1 + \frac{\Delta t F}{2 } \right) u^n \end{align}
\begin{align}
u^{n+1} = \left( 1 - \frac{ \Delta tF}{2} \right)^{-1} \left(1 + \frac{\Delta t F}{2 } \right)u^n
\end{align}
Taking into account the reaction equations for a diffusion reaction system 
\begin{align}
u^{n+1} &= \left( 1 - \frac{ \Delta t}{2 }D_u L \right)^{-1} \left(1 + \frac{\Delta t}{2 }D_uL  \right)u^n + 
\Delta t\ g(u^n)
\end{align}
where $L$ is the Laplacian operator. 

Note: we could bring the $u$ term into the linear operator when we use the Crank Nicolson method. 


In [2]:
# let's make some kind of domain
#https://docu.ngsolve.org/latest/netgen_tutorials/define_2d_geometries.html
geo = netgen.geom2d.SplineGeometry()
fac = 100
p1 = geo.AppendPoint (0,0)
p2 = geo.AppendPoint (0.7*fac,-0.3*fac)
p3 = geo.AppendPoint (1.4*fac,0*fac)
p4 = geo.AppendPoint (0.6*fac,1*fac)

geo.Append (["spline3", p1, p2, p3],bc="bottom")
geo.Append (["line", p3, p4], bc="right")
geo.Append (["line", p4, p1], bc="left")

mtemp = geo.GenerateMesh (maxh=2); Draw(mtemp) 
# lets you look at the mesh without making it into a full mesh that is suitable for FEM

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'mesh_dim': 2, 'mesh_cente…

BaseWebGuiScene

In [4]:
mesh = ngsolve.Mesh(geo.GenerateMesh (maxh=2))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [5]:
# the extra nonlinear function: u is the field, b_3 is the free parameter 
def g(u,b_3):
    return (u-(b_3 -1j)*np.absolute(u)*np.absolute(u)*u)   # Chate's form 

In [6]:
fes = ngsolve.H1(mesh, order=4, complex=True)  # We need a complex element
u, v = fes.TnT() # trial and test functions!
gfu = ngsolve.GridFunction(fes) # to hold the solution 


In [8]:
np.random.randint(10)

6

In [22]:
# create some random initial conditions!
def gen_init(gfu):
    sig = 1e-5
    nv = len(gfu.vec.FV().NumPy()[:])
    for k in range(nv):
        gfu.vec.FV().NumPy()[k] = 0.0 + 0*1j   # reset 
    rvec = np.random.uniform(size=nv)*sig + np.random.uniform(size=nv)*sig*1j/2
    gfu.vec.FV().NumPy()[:] = rvec
    #gfu.vec.FV().NumPy()[:] +=  np.random.uniform(size=nv)*sig*1j/2
    for k in range(30):
        j = np.random.randint(nv-1)
        gfu.vec.FV().NumPy()[j] = 1.0  # some seed values
        
gen_init(gfu)
Draw(gfu)  # with pull down menu you can choose real or complex parts to plot or the norm

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

BaseWebGuiScene

In [23]:
alpha = 2.  # must be defined here to make the Laplacian op    alpha = c_1 = b_1 of the different forms
D_u  = (1 + 1.j*alpha)  # complex diffusion coefficient , Chate form 
dt = .005    # timestep 

# Laplacian op
a = ngsolve.BilinearForm(fes, symmetric=True)  # holds Laplacian operator 
a += D_u*ngsolve.grad(u)*ngsolve.grad(v)*ngsolve.dx # This is laplacian with a complex diffusion coef 
a.Assemble()

# mass operator 
m = ngsolve.BilinearForm(fes)  # hold mass matrix for both fields 
m += u*v*ngsolve.dx 
m.Assemble()

print(f"m.mat.nze = {m.mat.nze}, a1.mat.nze={a.mat.nze}") # check that they are the same
# these are the number of nonzero elements in the sparse matrices

# B = M - 0.5*L*dt , needed for Crank Nicholson update L = Laplacian times diffusion coeff, M = mass matrix
b = m.mat.CreateMatrix()
b.AsVector().data     = m.mat.AsVector() - 0.5*dt * a.mat.AsVector()
print(f"b.nze = {b.nze}")

# A* = M + 0.5 * L * dt   needed for Crank Nicholson update 
astar = m.mat.CreateMatrix() # create a matrix in the form of m
astar.AsVector().data = m.mat.AsVector() + 0.5*dt * a.mat.AsVector()
invastar = astar.Inverse(freedofs=fes.FreeDofs())
print(f"astar.nze={astar.nze}")

# following everything we did previously for Brusselator in the notebook Brus_Circle.ipynb but now with a single and complex field

m.mat.nze = 891193, a1.mat.nze=891193
b.nze = 891193
astar.nze=891193


In [24]:
# arguments:
#  invastar, b :      # matrix operators on the finite element system 
#  nsamples:  number of outputs to store in the multidimensional data set 
#  b_3: parameter for the complex ginzburg landau model (chate form)
#  dt:        timestep  which is used in the matrix operators invastar and b 
# predefined things:
#  gfu      # for holding solution, is a Gridfunction on a predefined mesh
#  scene1,scene2    # for drawing both fields, outputs of Draw ngsolve.webgui
# returns 
#  gfut:   a multidimensional set of grid functions on finite element system that holds time stepped results
def TimeStepping_CN(invastar, b,  b_3, dt, initial_cond = None, t0 = 0, tend = 15, 
                 nsamples = 50):
    if initial_cond:
        gfu.Set(initial_cond)   # set initial condition, otherwise don't touch the initial fields 
        # as the initial condition could be set already in gfu.vec.data
    cnt = 0; time = t0
    sample_int = int(np.floor(tend / dt / nsamples)+1)  # nsamples is probably the number of outputs we want
    
    gfut = ngsolve.GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)  #  I think this makes it so we can store a series of solutions 

    while time < tend - 0.5 * dt:
        
        res =  b * gfu.vec   # you can just multiply b onto gfu.vec (it is a matrix multiply!) # if b is just a matrix
        # res is defined here and is now a vector on the finite element system 
        gfu.vec.data = invastar * res  # replacing solution here for Crank Nicholson update  
        # note you can just multiply invastar onto res  (this is a matrix multiply!)

        # operator split add in non-linear part by hand 
        upass = gfu.vec.FV().NumPy()[:]  # get the field 
        gfun  = g(upass,b_3) # compute the nonlinear term 
        gfu.vec.FV().NumPy()[:] += dt*gfun # add in non-linear function 
        # why I am doing this by hand?  NGsolve has non linear examples but I think they linearize and we want a fully 
        # non linear PDE here , we could try other techniques!
        
        print("\r",time,end="")
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
            scene.Redraw()
        cnt += 1; time = cnt * dt
    return gfut

In [27]:
# create some fresh random initial conditions! (again!)
#gfu = ngsolve.GridFunction(fes) # remake the field to hold the solution 
gen_init(gfu)
scene = Draw(gfu)  # with pull down menu you can choose real or complex parts to plot or the norm

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

In [28]:
b_3 = 1.3 # choose a nice pattern ?
gfut_CN = TimeStepping_CN(invastar, b,b_3, dt, tend=10)

 9.99500000000000153

In [ ]:
# seems to have worked.  It would be nice to show phase as well as norm, webgui is not helpful on this respect